In [7]:
"""This module contains functions to scrape all the activities listed on a city page."""
import time
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import openpyxl

In [9]:
def get_city_url_list(city_url, max_pages=100):
    """The city url is the first page of activities, most city urls have multiple pages
    of activities, so we need to check if the page exists and if it does, add it to
    a dictionary of valid pages. The key is the page number and the value is the url.
    The variable max_pages is the maximum number of pages to check, this is to prevent
    the scraper from running for too long if the city has a lot of pages."""

    response = requests.get(city_url, timeout=5)
    if response.status_code != 200:
        print(f"City url is not valid: {city_url}")
        return {}
    print(f"Checked validity for page 1 and it was 200")

    valid_pages = {1: response}
    if max_pages > 1:
        for page_number in range(2, max_pages+1):
            start = time.time()
            page_url = city_url + f"/{page_number}"
            response = requests.get(page_url, timeout=5)
            if response.status_code == 200:
                valid_pages[page_number] = response
            print(f"Checked validity for page {page_number} and it was {response.status_code}. Returned in {time.time() - start} seconds")
    print(f"Found {len(valid_pages)} valid pages")
    return valid_pages

In [10]:
def get_activity_urls(soup):
    """This function takes bs4 soup object of a city activity page and returns a list of
    activity urls contained in it."""

    activity_urls = []
    activity_links = soup.find_all("a", class_='text-dark highlight-able card-link')
    for link in activity_links:
        activity_urls.append("https://viator.com" + link['href'])
    return activity_urls

In [11]:
def extract_data(link):
    """This function takes a link to an activity page and returns a list
    of the data contained in it."""

    data = []
    response = requests.get(link, timeout=5)
    soup = bs(response.text, 'html.parser')


    activity_name = soup.find("h1", class_="title__1Wwg title2__C3R7").text
    activity_url = link
    price_per_participant = soup.find_all("span", class_="moneyView__2HPx defaultColor__1NL9")[0].text
    valid_number_of_participants = soup.find("input", class_ = "input__MNXR md__1Fp3")['data-automation-value']
    valid_date = soup.find("input", class_ = "input__2pmO md__2bZz")["value"]

    overview_block = soup.find("div", class_="overviewWrapper__bMs4")

    main_overview_ = overview_block.find_all("div")[0]
    try:
        activity_description = main_overview_.find_all("div")[0].text
    except:
        activity_description = "No description available"

    
    data.append(activity_name)
    data.append(activity_url)
    data.append(price_per_participant)
    data.append(valid_number_of_participants)
    data.append(valid_date)
    data.append(activity_description)

    return data

    

In [19]:
def main_scraper(city_url, max_pages=100, dataframe=pd.DataFrame(columns=['Activity Name', 'Activity URL', 'Price per Participant', 'Valid Number of Participants', 'Valid Date', 'Activity Description'])):
    """This function takes a city url and returns a dataframe of all the activities
    listed on the city page."""

    valid_pages = get_city_url_list(city_url, max_pages)
    activity_urls = []
    for page_number, response in valid_pages.items():
        start = time.time()
        soup = bs(response.text, 'html.parser')
        activity_urls += get_activity_urls(soup)
        print(f"Activity urls for page {page_number} added to list in {time.time() - start} seconds")
    print(f"Found {len(activity_urls)} activity urls")

    print(f"Estimated time to completion: {len(activity_urls) * 1.8} seconds")

    init_start = time.time()
    for link in activity_urls:
        start = time.time()
        try:
            data = extract_data(link)
            dataframe.loc[len(dataframe)] = data
            print(f"Extracted data for {link} in {time.time() - start} seconds. Estimated time to completion: {len(activity_urls) * 2.7 - (time.time() - init_start)} seconds")
        except:
            print(f"Error extracting data for {link}")
            continue
    print(f"Finished scraping {city_url}")
    return dataframe
    

In [20]:
if __name__ == "__main__":
    df = pd.DataFrame(columns=['Activity Name', 'Activity URL', 'Price per Participant', 'Valid Number of Participants', 'Valid Date', 'Activity Description'])
    df = main_scraper("https://www.viator.com/Bangkok/d343-ttd", max_pages = 21)
    df.to_excel("paris_activities.xlsx", index=False)

Checked validity for page 1 and it was 200
Found 1 valid pages
Activity urls for page 1 added to list in 0.2331087589263916 seconds
Found 24 activity urls
Estimated time to completion: 64.80000000000001 seconds
Extracted data for https://viator.com/tours/Bangkok/Train-Market-and-Damnoensaduak-Floating-Market-with-small-group/d343-8374P24 in 1.741882562637329 seconds. Estimated time to completion: 63.05811743736268 seconds
Extracted data for https://viator.com/tours/Bangkok/Amazing-Bangkok-City-and-Temple-Tour-with-Admission-Tickets/d343-91960P48 in 2.1555817127227783 seconds. Estimated time to completion: 60.902535724639904 seconds
Extracted data for https://viator.com/tours/Bangkok/Bangkok-by-Night-Food-Temples-and-Markets-by-Tuk-Tuk/d343-6467BKKNIGHT in 1.7150163650512695 seconds. Estimated time to completion: 59.187519359588634 seconds
Extracted data for https://viator.com/tours/Bangkok/Private-Tour-to-Ayutthaya-and-the-Summer-Palace/d343-52374P1 in 1.7964932918548584 seconds. Estim